In [3]:
# Cell 1: Install required packages (run this first)
!pip install language-tool-python pyaudio SpeechRecognition wave

  Using cached speechrecognition-3.14.2-py3-none-any.whl.metadata (30 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached speechrecognition-3.14.2-py3-none-any.whl (32.9 MB)
  Created wheel for wave: filename=wave-0.0.2-py3-none-any.whl size=1279 sha256=f2f5cd548357152b1eaefdbef7ec7a52907ff75e329e287347975ae621277a6e
  Stored in directory: c:\users\iamso\appdata\local\pip\cache\wheels\8b\ea\98\75ed94296181cd3404eb307bf6b8758bfc4a0a487e15493eb5
Successfully built wave


In [4]:
# Cell 2: Import libraries
import language_tool_python
from IPython.display import Audio, display
import speech_recognition as sr
import pyaudio
import wave
import os

In [5]:
# Cell 3: Grammar Analyzer Class
class GrammarAnalyzer:
    def __init__(self):
        self.tool = language_tool_python.LanguageTool('en-US')
    
    def analyze(self, text):
        if text.startswith("[Error]"):
            return 0, text, []
            
        matches = self.tool.check(text)
        error_count = len(matches)
        
        # Highlight errors
        highlighted = text
        for error in sorted(matches, key=lambda x: -x.offset):
            start = error.offset
            end = error.offset + error.errorLength
            highlighted = highlighted[:start] + f"[[{highlighted[start:end]}→{error.replacements[:3]}]]" + highlighted[end:]
        
        score = max(0, 100 - (error_count * 10))
        return score, highlighted, matches

In [6]:
# Cell 4: Audio Recorder Function
def record_audio(filename="recording.wav", duration=5):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000  # Lower sample rate for better speech recognition
    
    p = pyaudio.PyAudio()
    
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    
    print(f"Recording for {duration} seconds... (Speak now)")
    frames = []
    for _ in range(0, int(RATE / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)
    
    print("Finished recording")
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    
    return filename

In [7]:
# Cell 5: Audio Analysis Function
def analyze_audio(filename="recording.wav"):
    # Initialize recognizer
    recognizer = sr.Recognizer()
    
    # Verify file exists
    if not os.path.exists(filename):
        print(f"Error: File '{filename}' not found")
        return
    
    # Play the audio
    display(Audio(filename))
    
    # Transcribe audio
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"Transcribed Text: {text}")
            
            # Analyze grammar
            analyzer = GrammarAnalyzer()
            score, highlighted, matches = analyzer.analyze(text)
            
            print(f"\nGrammar Score: {score}/100")
            print(f"Corrected Text: {highlighted}")
            
            if matches:
                print("\nError Details:")
                for i, match in enumerate(matches[:5], 1):
                    print(f"{i}. At position {match.offset}:")
                    print(f"   Error: '{text[match.offset:match.offset+match.errorLength]}'")
                    print(f"   Suggested: {match.replacements[:3]}")
                    print(f"   Rule: {match.ruleId} - {match.message}")
                    
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")

In [8]:
# Cell 6: Demo Usage
print("=== Grammar Scoring Engine ===")
print("1. First, let's record your voice")
record_audio()  # Records for 5 seconds

print("\n2. Now analyzing your recording...")
analyze_audio()  # Analyzes the recording.wav file

=== Grammar Scoring Engine ===
1. First, let's record your voice
Recording for 5 seconds... (Speak now)
Finished recording

2. Now analyzing your recording...


Transcribed Text: calling for 5 second speaker


ModuleNotFoundError: No java install detected. Please install java to use language-tool-python.